# Timesheet Utility Notebook
This notebook loads the SAP Excel file, extracts relevant data, and structures it in a DataFrame.

### Install Required Packages

In [24]:
# 📦 Ensure required packages are installed
import subprocess
import sys

def install_if_missing(pip_name, import_name=None):
    import_name = import_name or pip_name
    try:
        __import__(import_name)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])

install_if_missing("pandas")
# Install the openpyxl package
install_if_missing("openpyxl")
install_if_missing("xlsxwriter")

### Import Required Libraries

In [25]:
import re
import os
import math
import datetime
import pandas as pd

from io import BytesIO
from pathlib import Path

### Load and preprocess SAP timesheet data
This loads the SAP Excel file, extracts relevant data, and structures it in a DataFrame.

In [26]:
# Load the Excel file
file_path = Path("reports") / "sap.xlsx"
xls = pd.ExcelFile(file_path, engine="openpyxl")

# Parse the sheet
df_raw = xls.parse('Sheet1', header=None)

# Extract actual headers from row 18 and data from row 19 onwards
df_data = df_raw.iloc[19:].copy()
df_data.columns = df_raw.iloc[18]

# Read the cell that contains the date range (assuming it's in row index 8 and column 1)
date_range_str = str(df_raw.iloc[8, 1])

# Use regex to extract both dates
match = re.search(r"\[(\d{2}\.\d{2}\.\d{4})\s*-\s*(\d{2}\.\d{2}\.\d{4})\]", date_range_str)
if match:
    start_date = datetime.datetime.strptime(match.group(1), "%d.%m.%Y")
    end_date = datetime.datetime.strptime(match.group(2), "%d.%m.%Y")
    print("Start Date:", start_date)
    print("End Date:", end_date)
else:
    raise ValueError("Date range not found in the expected format.")

# Forward fill the project cost center column where values are missing
df_data['Project Cost Center'] = df_data['Project Cost Center'].ffill()

# Select relevant columns
sap_df = df_data[['Employee Email Address', 'Date', 'Hours','Project Cost Center']].copy()
# Filter SAP by date range
sap_df = sap_df[(sap_df['Date'] >= start_date) & (sap_df['Date'] <= end_date)]
sap_df.rename(columns={'Employee Email Address': 'Email'}, inplace=True)

# Clean the Hours column (remove ' H' and convert to float)
sap_df['Hours'] = pd.to_numeric(
    sap_df['Hours'].astype(str).str.replace(' H', '', regex=False),
    errors='coerce'
)

# Reset index
sap_df.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print("SAP data loaded successfully.")
print("First few rows of the DataFrame:")
sap_df.head()

Start Date: 2025-07-01 00:00:00
End Date: 2025-07-31 00:00:00
SAP data loaded successfully.
First few rows of the DataFrame:


18,Email,Date,Hours,Project Cost Center,Project Cost Center
0,ishita.bhown@nagarro.com,2025-07-01 00:00:00,8,US1S0044,CliTag-US10
1,ishita.bhown@nagarro.com,2025-07-02 00:00:00,8,US1S0044,CliTag-US10
2,ishita.bhown@nagarro.com,2025-07-03 00:00:00,8,US1S0044,CliTag-US10
3,ishita.bhown@nagarro.com,2025-07-04 00:00:00,8,US1S0044,CliTag-US10
4,ishita.bhown@nagarro.com,2025-07-07 00:00:00,8,US1S0044,CliTag-US10


### Load and preprocess WAND timesheet data
This notebook loads the WAND Excel file, extracts the relevant data, and structures it in a clean DataFrame.

In [27]:
# Load the Excel file
file_path = Path("reports") / "wand.xlsx"
df_raw = pd.read_excel(file_path, engine="openpyxl", header=None)

# Extract header row and data
df_data = df_raw.iloc[3:].copy()
df_data.columns = df_raw.iloc[2]
df_data.columns = df_data.columns.str.strip()

# Select relevant columns
wand_df = df_data[['Worker', 'Date', 'Hours']].copy()

# Filter WAND by date range
wand_df = wand_df[(wand_df['Date'] >= start_date) & (wand_df['Date'] <= end_date)]

# Convert Hours to float
wand_df['Hours'] = wand_df['Hours'].astype(str).str.replace(' H', '', regex=False).astype(float)

# Reset index
wand_df.reset_index(drop=True, inplace=True)

# Display final DataFrame
print("SAP data loaded successfully.")
print("First few rows of the DataFrame:")
wand_df.head()

SAP data loaded successfully.
First few rows of the DataFrame:


/Users/satyamvats/Desktop/Timesheet_Utility/.venv/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


2,Worker,Date,Hours
0,"Sharma, Divya",2025-07-01 00:00:00,8.0
1,"Sharma, Divya",2025-07-02 00:00:00,8.0
2,"Sharma, Divya",2025-07-03 00:00:00,8.0
3,"Sharma, Divya",2025-07-04 00:00:00,8.0
4,"Aggarwal, Ajay",2025-07-01 00:00:00,8.0


### Map WAND Worker to SAP Email using Mapping File
This notebook loads WAND timesheet data and a mapping file, then maps the WAND Worker to the corresponding SAP email address.

### Load and clean the mapping file

In [28]:
# Load the mapping file directly with headers
file_path = Path("reports") / "mapping.xlsx"
mapping_df = pd.read_excel(file_path, engine="openpyxl")

# Strip whitespace from column names
mapping_df.columns = mapping_df.columns.str.strip()

# Reset index (optional)
mapping_df.reset_index(drop=True, inplace=True)

# Display result
print("Mapping data loaded successfully.")
mapping_df.head()

Mapping data loaded successfully.


,emailAddress,proWandName,projectName,projectType
0,abhishek.singh19@nagarro.com,"Singh, Abhishek",CPAT,TNM
1,ajay.aggarwal@nagarro.com,"Aggarwal, Ajay",CPAT,TNM
2,akash.barman@nagarro.com,NaN,CPAT,SOW
3,aman.sharma07@nagarro.com,NaN,CPAT,SOW
4,amit.bindal01@nagarro.com,"Bindal, Amit",CPAT,TNM


### Formatting and Writing the Final Report

### Define function to generate a summary sheet

In [29]:
def format_comparison_sheet(workbook, worksheet, merged_df):
    header_format = workbook.add_format({
        'bold': True, 'text_wrap': True, 'valign': 'center',
        'fg_color': '#DDEBF7', 'border': 1
    })
    number_format = workbook.add_format({'num_format': '0.00', 'border': 1})
    text_format = workbook.add_format({'border': 1})
    highlight_format = workbook.add_format(
        {'bg_color': '#FFD6D6', 'border': 1})
    date_format = workbook.add_format(
        {'num_format': 'dd-mmm-yyyy', 'border': 1})

    worksheet.set_column('A:A', 15)
    worksheet.set_column('B:B', 30)
    worksheet.set_column('C:C', 25)
    worksheet.set_column('D:D', 20)
    worksheet.set_column('E:G', 12)

    for col_num, value in enumerate(merged_df.columns):
        worksheet.write(0, col_num, value, header_format)

    for row_num in range(1, len(merged_df) + 1):
        for col_num, col_name in enumerate(merged_df.columns):
            value = merged_df.iloc[row_num - 1, col_num]

            # === Safely handle NaN and inf ===
            if pd.isna(value) or (isinstance(value, (float, int)) and math.isinf(value)):
                value = None  # or 'N/A' if you prefer

            # === Choose format ===
            if col_name == 'Date':
                fmt = date_format
            elif col_name.startswith('Hours') or col_name == 'Delta':
                fmt = highlight_format if col_name == 'Delta' and value not in [0, None] else number_format
            else:
                fmt = text_format

            worksheet.write(row_num, col_num, value, fmt)


def write_summary_sheet(writer, merged_df, mapping_df, email_col):
    summary_data = []

    for email in merged_df['Email'].dropna().unique():
        df = merged_df[merged_df['Email'] == email]

        # Non-zero days only
        sap_dates = set(df[df['Hours_sap'] > 0]['Date'])
        wand_dates = set(df[df['Hours_wand'] > 0]['Date'])

        extra_sap = sorted([d.strftime("%d") for d in (sap_dates - wand_dates)])
        extra_wand = sorted([d.strftime("%d") for d in (wand_dates - sap_dates)])

        # Dates where both have entries but hours differ
        mismatched_dates = sorted([
            d.strftime("%d")
            for d in (sap_dates & wand_dates)   # intersection (both filled)
            if df.loc[df['Date'] == d, 'Hours_sap'].sum() != df.loc[df['Date'] == d, 'Hours_wand'].sum()
        ])

        total_sap = df['Hours_sap'].sum()
        total_wand = df['Hours_wand'].sum()
        delta = total_sap - total_wand

        full_name = df['Full Name'].iloc[0] if 'Full Name' in df else ""
        team = mapping_df[mapping_df[email_col] == email]['projectName'].iloc[0] if email in mapping_df[email_col].values else "N/A"
        
        status = "OK" if (df['Delta'] == 0).all() else "NOT OK"

        summary_data.append({
            "Email": email,
            "Full Name": full_name,
            "Days Only in SAP": extra_sap,
            "Days Only in WAND": extra_wand,
            "Mismatched Days": mismatched_dates,   # ✅ new column
            "Total SAP Hours": round(total_sap, 2),
            "Total WAND Hours": round(total_wand, 2),
            "Hour Difference": round(delta, 2),
            "Team": team,
            "Status": status
        })

    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, index=False, sheet_name='Summary')
    return summary_df


def format_summary_sheet(workbook, worksheet, summary_df):
    header_format = workbook.add_format({
        'bold': True, 'text_wrap': True, 'valign': 'center',
        'fg_color': '#DDEBF7', 'border': 1
    })

    worksheet.set_column('A:A', 30)  # Email
    worksheet.set_column('B:B', 25)  # Full Name
    worksheet.set_column('C:E', 30)  # Extra Days + Mismatched
    worksheet.set_column('F:H', 18)  # Totals & Diff
    worksheet.set_column('I:I', 15)  # Team
    worksheet.set_column('J:J', 15)  # Status

    for col_num, value in enumerate(summary_df.columns):
        worksheet.write(0, col_num, value, header_format)
    
    # Conditional formatting for Status
    worksheet.conditional_format(f'J2:J{len(summary_df)+1}', {
        'type':     'text',
        'criteria': 'containing',
        'value':    'NOT OK',
        'format':   workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
    })

    worksheet.conditional_format(f'J2:J{len(summary_df)+1}', {
        'type':     'text',
        'criteria': 'containing',
        'value':    'OK',
        'format':   workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
    })


def format_chart_sheet(workbook, sheets, summary_df):
    worksheet = workbook.add_worksheet('Charts')

    # Add chart 1: Bar chart of SAP vs WAND hours
    bar_chart = workbook.add_chart({'type': 'column'})

    row_count = len(summary_df)

    bar_chart.add_series({
        'name': 'SAP Hours',
        'categories': ['Summary', 1, 0, row_count, 0],  # Email column
        'values':     ['Summary', 1, 5, row_count, 5],  # Total SAP Hours
        'fill':       {'color': '#5B9BD5'},
    })
    bar_chart.add_series({
        'name': 'WAND Hours',
        'categories': ['Summary', 1, 0, row_count, 0],
        'values':     ['Summary', 1, 6, row_count, 6],  # Total WAND Hours
        'fill':       {'color': '#ED7D31'},
    })

    bar_chart.set_title({'name': 'SAP vs WAND Hours'})
    bar_chart.set_x_axis({'name': 'Email'})
    bar_chart.set_y_axis({'name': 'Hours'})
    bar_chart.set_style(11)

    worksheet.insert_chart('B2', bar_chart, {'x_scale': 1.5, 'y_scale': 1.5})

    # Add chart 2: Line chart of Hour Difference
    line_chart = workbook.add_chart({'type': 'line'})

    line_chart.add_series({
        'name': 'Hour Difference',
        'categories': ['Summary', 1, 0, row_count, 0],  # Email column
        'values':     ['Summary', 1, 7, row_count, 7],  # Hour Difference
        'line':       {'color': 'red'},
    })

    line_chart.set_title({'name': 'Hour Difference Per Employee'})
    line_chart.set_x_axis({'name': 'Email'})
    line_chart.set_y_axis({'name': 'Difference'})
    line_chart.set_style(10)

    worksheet.insert_chart('B20', line_chart, {'x_scale': 1.5, 'y_scale': 1.5})

### Export the final formatted report to Excel

In [30]:
def write_timesheet_report_to_excel(final_df: pd.DataFrame, mapping_df: pd.DataFrame, email_col: str = "Email", output_path: str = "final_timesheet_report.xlsx") -> None:
    """
    Write a formatted timesheet comparison report to Excel.

    Args:
        final_df (pd.DataFrame): The processed final comparison dataframe.
        mapping_df (pd.DataFrame): The mapping dataframe for emails and project names.
        email_col (str): The column name in final_df used as email identifier.
        output_path (str): Path to save the final Excel file.

    Returns:
        None
    """
    output = BytesIO()

    with pd.ExcelWriter(output, engine='xlsxwriter', datetime_format='dd-mmm-yyyy') as writer:
        # Sheet 1: Comparison Report
        final_df.to_excel(writer, index=False, sheet_name='Comparison Report')
        worksheet = writer.sheets['Comparison Report']

        worksheet.set_zoom(130)
        worksheet.freeze_panes(1, 0)
        worksheet.set_selection('A2')
        format_comparison_sheet(writer.book, worksheet, final_df)

        # Sheet 2: Summary Sheet
        summary_df = write_summary_sheet(writer, final_df, mapping_df, email_col=email_col)
        summary_ws = writer.sheets['Summary']
        format_summary_sheet(writer.book, summary_ws, summary_df)

        # Sheet 3: Charts
        format_chart_sheet(writer.book, writer.sheets, summary_df)

    output.seek(0)

    with open(output_path, "wb") as f:
        f.write(output.read())

    print(f"✅ Excel report saved as '{output_path}'")

### Combine SAP and WAND Data
This notebook combines the SAP and WAND timesheet data, calculates the difference in hours, and generates a formatted Excel report with a summary and charts.

In [31]:
def prepare_final_dataframe(sap_df: pd.DataFrame, wand_df: pd.DataFrame, mapping_df: pd.DataFrame) -> pd.DataFrame:
    # === Clean column names ===
    sap_df.columns = sap_df.columns.str.strip()
    wand_df.columns = wand_df.columns.str.strip()
    mapping_df.columns = mapping_df.columns.str.strip()

    # === Clean Mapping ===
    mapping_df = mapping_df.drop_duplicates(subset=["emailAddress", "proWandName"])
    mapping_df = mapping_df[mapping_df['projectType'].isin(['TNM', 'MC'])].copy()
    email_project_map = mapping_df.set_index("emailAddress")["projectName"].to_dict()
    valid_emails = set(mapping_df['emailAddress'])

    # === Process SAP ===
    sap_df = sap_df.rename(columns={'Email': 'emailAddress'})
    sap_df['Date'] = pd.to_datetime(sap_df['Date'], dayfirst=True).dt.date
    sap_df['projectName'] = sap_df['emailAddress'].map(email_project_map)

    # Keep only SAP entries for emails in mapping
    sap_df = sap_df[sap_df['emailAddress'].isin(valid_emails)]
    sap_grouped = sap_df.groupby(['emailAddress', 'Date'], as_index=False)['Hours'].sum()
    sap_grouped = sap_grouped.rename(columns={'Hours': 'Hours_sap'})

    # === Process WAND ===
    wand_df = wand_df.rename(columns={'Worker': 'WorkerName'})
    wand_df['Date'] = pd.to_datetime(wand_df['Date']).dt.date

    # Only include WAND entries that match mapping (inner join)
    wand_df = wand_df.merge(
        mapping_df[['emailAddress', 'proWandName', 'projectName']],
        how='inner',
        left_on='WorkerName',
        right_on='proWandName'
    )
    wand_grouped = wand_df.groupby(['emailAddress', 'Date'], as_index=False)['Hours'].sum()
    wand_grouped = wand_grouped.rename(columns={'Hours': 'Hours_wand'})

    # === Merge SAP and WAND ===
    combined = pd.merge(sap_grouped, wand_grouped, on=['emailAddress', 'Date'], how='outer')
    combined = combined.merge(mapping_df[['emailAddress', 'proWandName', 'projectName']], on='emailAddress', how='left')

    # === Final Cleanup ===
    combined['Hours_sap'] = combined['Hours_sap'].fillna(0)
    combined['Hours_wand'] = combined['Hours_wand'].fillna(0)
    combined['Delta'] = combined['Hours_sap'] - combined['Hours_wand']
    combined['Date'] = pd.to_datetime(combined['Date'])
    combined = combined.drop_duplicates(subset=['emailAddress', 'Date'])

    # === Format Final Output ===
    combined = combined.rename(columns={'emailAddress': 'Email'})
    combined['Full Name'] = combined['proWandName']
    final_df = combined[[
        'Date', 'Email', 'Full Name', 'proWandName', 'projectName',
        'Hours_sap', 'Hours_wand', 'Delta'
    ]].sort_values(by=['Email', 'Date'])

    return final_df

### Final Formatting and Writing to Excel
This notebook formats the combined timesheet data, generates a summary sheet, and writes the final report

In [32]:
final_df = prepare_final_dataframe(sap_df, wand_df, mapping_df)
final_file_path = os.path.join("reports", "final_timesheet_report.xlsx")
write_timesheet_report_to_excel(final_df, mapping_df, email_col="emailAddress", output_path=final_file_path)
print("✅ Report generated successfully.")

✅ Excel report saved as 'reports/final_timesheet_report.xlsx'
✅ Report generated successfully.
